In [9]:
#Question 1

# Importing the necessary packages
import math

# Defining the input parameters
inp = {
    "n-tosses": 0, # Number of coin tosses
    "stock_price": 0, # Initial stock price
    "strike_price": 0, # Strike price agreed upon
    "up_factor": 0, # Up factor constant
    "down_factor": 0, # Down factor constant
    "interest_rate": 0 # Interest rate in the money market
}

stonks = [] # Initializing the stock price array
option_prices = [] # Initializing an array to store the call option prices
p_t = 0 # Initializing risk neutral probabilities
q_t = 0 
v_r = 0 # Initializing call option price

def print_stonks(stonks): # This function prints the predicted stock prices
    for i in range(len(stonks)):
        print(stonks[i])

def cal_stock_prices(depth): # Driver code to predict stock prices
    global stonks 
    depth = int(depth)
    stonks = [{'': inp['stock_price']}] # Taking the input parameters 
    for i in range(depth): 
        stonks.append({})
        for key in stonks[i].keys():  
            stonks[i+1][key+'H'] = stonks[i][key] * inp["up_factor"]
            stonks[i+1][key+'T'] = stonks[i][key] * inp["down_factor"]
    return stonks

def cal_option_prices(): # Driver code to calculate the call option prices
    global option_prices
    option_prices = [{}]
    for key in stonks[inp['n-tosses']].keys(): # Calculating the final time option price
        option_prices[0][key] = max(stonks[inp['n-tosses']][key] - inp['strike_price'], 0)
    for i in range(0, inp['n-tosses']): 
        option_prices.append({}) # Appending to the option price array
        for key in option_prices[i].keys():
            if key[len(key)-1] == "H": # Printing the option prices
                print("vnH:", option_prices[i][key], "vnT:", option_prices[i][key[0:len(key)-1]+"T"])
                vn_h = (p_t * option_prices[i][key])
                vn_t = (q_t * option_prices[i][key[0:len(key)-1]+"T"])
                option_prices[i+1][key[0:len(key)-1]] = v_r * (vn_h + vn_t) # Recursively calculating the initial option price

def receive_input(): # This function feeds the input into another function
    for key in inp.keys():
        inp[key] = float(input("Enter value for " + str(key) + ">> "))


receive_input() # Accepting the input
p_t = float(1+inp['interest_rate']-inp['down_factor'])/(inp['up_factor'] - inp['down_factor']) # Calculating the risk neutral probabilities
q_t = 1-p_t
v_r = (1/(1+inp['interest_rate'])) # Calculating the option price
print(p_t, q_t, v_r) # Printing the risk neutral probabilities and the option prices

inp['n-tosses'] = int(inp['n-tosses'])
print_stonks(cal_stock_prices(inp['n-tosses'])) # Printing the input
cal_option_prices() 
print("\ntentative values") # Printing the option prices
print_stonks(option_prices)

3.6666666666666665 -2.6666666666666665 0.16666666666666666
{'': 10.0}
{'H': 20.0, 'T': 5.0}
{'HH': 40.0, 'HT': 10.0, 'TH': 10.0, 'TT': 2.5}
{'HHH': 80.0, 'HHT': 20.0, 'HTH': 20.0, 'HTT': 5.0, 'THH': 20.0, 'THT': 5.0, 'TTH': 5.0, 'TTT': 1.25}
{'HHHH': 160.0, 'HHHT': 40.0, 'HHTH': 40.0, 'HHTT': 10.0, 'HTHH': 40.0, 'HTHT': 10.0, 'HTTH': 10.0, 'HTTT': 2.5, 'THHH': 40.0, 'THHT': 10.0, 'THTH': 10.0, 'THTT': 2.5, 'TTHH': 10.0, 'TTHT': 2.5, 'TTTH': 2.5, 'TTTT': 0.625}
{'HHHHH': 320.0, 'HHHHT': 80.0, 'HHHTH': 80.0, 'HHHTT': 20.0, 'HHTHH': 80.0, 'HHTHT': 20.0, 'HHTTH': 20.0, 'HHTTT': 5.0, 'HTHHH': 80.0, 'HTHHT': 20.0, 'HTHTH': 20.0, 'HTHTT': 5.0, 'HTTHH': 20.0, 'HTTHT': 5.0, 'HTTTH': 5.0, 'HTTTT': 1.25, 'THHHH': 80.0, 'THHHT': 20.0, 'THHTH': 20.0, 'THHTT': 5.0, 'THTHH': 20.0, 'THTHT': 5.0, 'THTTH': 5.0, 'THTTT': 1.25, 'TTHHH': 20.0, 'TTHHT': 5.0, 'TTHTH': 5.0, 'TTHTT': 1.25, 'TTTHH': 5.0, 'TTTHT': 1.25, 'TTTTH': 1.25, 'TTTTT': 0.3125}
vnH: 305.0 vnT: 65.0
vnH: 65.0 vnT: 5.0
vnH: 65.0 vnT: 5.0
vn

In [ ]:
#Question 2
import numpy as np

def black_scholes_put(S, K, r, sigma, T, N, M):
    """Solves the Black-Scholes-Merton PDE for a European put option using the implicit finite difference method.
    
    Parameters
    ----------
    S : float
        Initial stock price.
    K : float
        Strike price.
    r : float
        Risk-free interest rate.
    sigma : float
        Volatility of the stock.
    T : float
        Time to expiration in years.
    N : int
        Number of time steps.
    M : int
        Number of stock price steps.
    
    Returns
    -------
    V : numpy array
        The option value at each time step and stock price step.
    """
    # Set up the grid
    dt = T / N
    dx = sigma * np.sqrt(dt)
    u = np.exp(dx)
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    V = np.empty((N+1, M+1))
    
    # Set up the boundary conditions
    for j in range(M+1):
        V[N, j] = max(K - S * u*j * d*(M - j), 0)
    for i in range(N, -1, -1):
        for j in range(M, -1, -1):
            V[i, j] = max(np.exp(-r * dt) * (p * V[i+1, j+1] + (1 - p) * V[i+1, j]), K - S * u*j * d*(M - j))
    
    return V

V = black_scholes_put(50, 50, 0.05, 0.2, 1, 100, 50)